[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ElMartinez31/Data_Science/blob/main/Projects/Classical_Frameworks/Riot_Games_Data_Scrapping.ipynb)

In [2]:
import pandas as pd
import numpy as np
!pip install requests pandas


L'idée est de trvailler vec l'API de Riot Games pour analyser leur données, et voir au fil de l'eau quel genre de stats on peut en faire.

In [ ]:
# Documentation:
# Chaque section permet de récupérer différents types de données.
# Les données sont bien structurées, généralement des imbrications de dictionnaires.

#https://developer.riotgames.com/apis#tft-league-v1/GET_getGrandmasterLeague

In [52]:

import requests

API_KEY = "RGAPI-2d61e96c-b57d-4b5e-a43e-a904cff43420" # Temporary (24h)
region = "europe"  # Notez que pour l'endpoint match-v5, vous devez utiliser "europe" pour EU

puuid = "zukEKR0z2EyB3c_LczgLeXVjPCslbQySn4FwqdM4LuZwVbVUEBauk0FQY7huJYiUWhIx1m67C7FPkg"

match_list_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={API_KEY}"

try:
    match_response = requests.get(match_list_url)
    match_response.raise_for_status()
    match_ids = match_response.json()

    print(f"Nombre de matches trouvés : {len(match_ids)}")
    print("Liste des IDs de match :")
    print(match_ids)

    # Si vous voulez obtenir les détails d'un match spécifique
    if match_ids:
        match_id = match_ids[0]  # Prend le premier ID de match
        match_details_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={API_KEY}"
        match_details_response = requests.get(match_details_url)
        match_details_response.raise_for_status()
        match_details = match_details_response.json()

        print(f"Détails du premier match :")
        print(match_details)

except requests.exceptions.HTTPError as errh:
    print(f"Erreur HTTP: {errh}")
except Exception as e:
    print(f"Une erreur est survenue : {e}")

Nombre de matches trouvés : 20
Liste des IDs de match :
['EUW1_7278716352', 'EUW1_7265682177', 'EUW1_7221922700', 'EUW1_7221912813', 'EUW1_7213798836', 'EUW1_7213762200', 'EUW1_7213678983', 'EUW1_7207989524', 'EUW1_7207685185', 'EUW1_7207622109', 'EUW1_7194082265', 'EUW1_7187847640', 'EUW1_7185899435', 'EUW1_7185839260', 'EUW1_7183591170', 'EUW1_7182556796', 'EUW1_7182533820', 'EUW1_7182511299', 'EUW1_7182493598', 'EUW1_7182440262']
Détails du premier match :
{'metadata': {'dataVersion': '2', 'matchId': 'EUW1_7278716352', 'participants': ['JraFwLD6sYXFx908ACiFXHajianIb88kgwT29s44ezIzlI8zYebzVbXjLYSwrsAzzTvs8H2kb74E8A', 'RejGoAYaX_er5BxbJpS9huBhZ3kz3ASweUgRS0jxBNZ9EaJGwNJGbMhtoKNuzoRW_qZspCioUadgdw', '5yArqXK_6NhANjlzGqOCxsV9NyyqUWpvSpB5480bOGwAdkiSvvShmO9yAOEx8ZFQRP7Lm4Saag0jaw', '8Ahl5w6pa38uqsPpCwBfvay-FCKaKPJ51tPD3cNFp_7qmtecczkmM2f6UtZyh3G9vf6OOf6WkT8m4A', 'rW6TTCNxXtIMgkPgULstTVm_69oYkSB3XdtlM_sru4dLSr5IbtIpMs_4kIZx51Hw9ByGVDdAKFvJDw', 'HAHK7UfJ07nKZF5auQYteBYTLMIEvjxk2Iw89iUP1rqT

In [54]:
import pandas as pd
from pandas import json_normalize


metadata_df = json_normalize(match_details['metadata'])
print("DataFrame Metadata:")
metadata_df

DataFrame Metadata:


,dataVersion,matchId,participants
0,2,EUW1_7278716352,[JraFwLD6sYXFx908ACiFXHajianIb88kgwT29s44ezIzl...


In [55]:
info_df = json_normalize(match_details['info'])
print("DataFrame Info:")
info_df

DataFrame Info:


,endOfGameResult,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,participants,platformId,queueId,teams,tournamentCode
0,GameComplete,1737661753307,1745,1737663528036,7278716352,CLASSIC,teambuilder-match-7278716352,1737661783085,MATCHED_GAME,15.2.650.9216,11,"[{'PlayerScore0': 0, 'PlayerScore1': 0, 'Playe...",EUW1,420,"[{'bans': [{'championId': -1, 'pickTurn': 1}, ...",


In [56]:
API_KEY = "RGAPI-2d61e96c-b57d-4b5e-a43e-a904cff43420"
url = "https://euw1.api.riotgames.com/tft/league/v1/grandmaster?queue=RANKED_TFT&api_key=" + API_KEY

try:
    response = requests.get(url)
    response.raise_for_status()
    json_data = response.json()

except requests.exceptions.HTTPError as errh:
    print(f"Erreur HTTP: {errh}")
except Exception as e:
    print(f"Une erreur est survenue : {e}")

In [57]:
# Convertir la liste des entrées en DataFrame
entries_df = json_normalize(json_data['entries'])
print("DataFrame des joueurs Grandmaster:")
print(entries_df.head())

# Vous pouvez également vouloir conserver ou afficher d'autres informations du JSON
# comme la date de mise à jour de la ligue, le nom de la ligue, etc.
league_info = {
    'leagueId': json_data.get('leagueId'),
    'tier': json_data.get('tier'),
    'queue': json_data.get('queue'),
    'name': json_data.get('name'),
    'entries': len(json_data.get('entries', []))
}
league_df = pd.DataFrame([league_info])
print("\nInformations de la ligue:")
print(league_df)

DataFrame des joueurs Grandmaster:
                                          summonerId  leaguePoints rank  wins  \
0  Aax4UksYkdB4H0AXBsu9P1YD3qJnwgWL9P36mzkgvuei4i...          1160    I   180   
1    j2HkrVlDZDPEUVEboLVbp4znCNJhUrrzQuLd-AhVy-J1R_w          1097    I   347   
2  9Hx_FQcs72LkTrGzJYIgrrk2ovVB50pj_Uo9Kr6XafJgaK...          1066    I    94   
3  qjJ4aSfY8JKXuXvBdbIXVCQ0C7PvrpwCJv4dkdRju7ntrE...          1060    I   335   
4   e9eIaiWgE6HR5PcJZBUcr3bcUh3tRs-98hPpW-cTy745cq0L          1052    I   573   

   losses  veteran  inactive  freshBlood  hotStreak  
0     132    False     False        True      False  
1     293    False     False        True       True  
2      58    False     False        True       True  
3     277    False     False        True      False  
4     573    False     False       False      False  

Informations de la ligue:
                               leagueId         tier       queue  \
0  b91b91ba-8e25-3a6c-b8a4-b144602fae64  GRANDMASTER  RANKE